Reference Notebook
https://colab.research.google.com/github/coqui-ai/TTS/blob/main/notebooks/Tutorial_2_train_your_first_TTS_model.ipynb#scrollTo=014879b7-f18d-44c0-b24a-e10f8002113a

In [1]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

#//*** create model output path if needed
output_path = "tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

In [2]:
output_path = "tts_train_dir"
output_path = os.path.join(os.path.join(os.getcwd(), output_path))
output_path

t_path = str(output_path).replace("\\","/")
t_path
#output_path = "C:\\Users\\stonk013\\KGO_Projects\\tts\\recipes\\ljspeech\\LJSpeech-1.1\\"
#output_path

'C:/Users/stonk013/KGO_Projects/tts/ama_model/tts_train_dir'

In [3]:
import os
import wave
fulldir = os.path.join(output_path,"wavs")
#//**** Get the sample rate of the first WAV file, assume all files share the same sample_rate
for file_name in os.listdir(fulldir):
    with wave.open(os.path.join(fulldir,file_name), "rb") as wave_file:
        frame_rate = wave_file.getframerate()
        print(file_name,frame_rate)
        break

ama-01.wav 48000


In [4]:
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples


# dataset config for one of the pre-defined datasets
dataset_config = BaseDatasetConfig(
    #formatter="vctk", meta_file_train="", language="en-us", path=os.path.join(os.path.join(output_path,"wavs"))
    formatter="ljspeech", meta_file_train="metadata.csv", language="en-us", path=t_path
)

# load training samples
train_samples, eval_samples = load_tts_samples(dataset_config, eval_split=False)

 | > Found 12 files in C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir


In [5]:
dataset_config


BaseDatasetConfig(formatter='ljspeech', dataset_name='', path='C:/Users/stonk013/KGO_Projects/tts/ama_model/tts_train_dir', meta_file_train='metadata.csv', ignored_speakers=None, language='en-us', phonemizer='', meta_file_val='', meta_file_attn_mask='')

In [6]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
    
)

from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
ap.sample_rate = frame_rate

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [7]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [8]:

from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=False,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 12 files in C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir


In [9]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [10]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Num. of CPUs: 6
 | > Num. of Torch Threads: 6
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-08-2023_11+39PM-0000000

 > Model has 28610257 parameters


In [11]:
trainer.fit()


 > EPOCH: 0/100
 --> C:\Users\stonk013\KGO_Projects\tts\ama_model\tts_train_dir\run-August-08-2023_11+39PM-0000000

 > TRAINING (2023-08-08 23:39:22) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 12
 | > Preprocessing samples
 | > Max text length: 908
 | > Min text length: 106
 | > Avg text length: 392.8181818181818
 | 
 | > Max audio length: 4811245.0
 | > Min audio length: 645483.0
 | > Avg audio length: 2134389.1818181816
 | > Num. instances discarded samples: 1
 | > Batch group size: 0.


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:/Users/stonk013/KGO_Projects/tts/ama_model/tts_train_dir/run-August-08-2023_11+39PM-0000000\\trainer_0_log.txt'

In [ ]:


import glob, os
#output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])



In [ ]:
tensorboard --logdir="tts_train_dir"